In [1]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Function to authenticate and get the Google Sheets client
def authenticate_google_sheets(json_credentials_path):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)
    return client

# Function to fetch data from a worksheet and convert it to a DataFrame
def fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path):
    try:
        # Authenticate and get the Google Sheets client
        client = authenticate_google_sheets(json_credentials_path)
        print("Authenticated successfully")

        # Open the existing Google Sheet by name
        sheet = client.open(sheet_name)
        print(f"Opened Google Sheet: {sheet_name}")

        # Get the worksheet by name
        worksheet = sheet.worksheet(worksheet_name)
        print(f"Accessing worksheet: {worksheet_name}")

        # Fetch all data from the worksheet
        data = worksheet.get_all_values()
        print("Fetched data from worksheet")

        # Convert the data to a DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])
        print("Converted data to DataFrame")

        return df

    except gspread.exceptions.APIError as api_error:
        print(f"Google Sheets API error: {api_error}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage
if __name__ == "__main__":
    # Path to the JSON credentials file
    json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

    # Name of the existing Google Sheet
    sheet_name = 'Tracking Sheet for Charlotte.xlsx'

    # Name of the worksheet
    worksheet_name = 'Test'

    # Fetch data from the worksheet into a DataFrame
    df_old = fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path)

    if df_old is not None:
        print("Data fetched successfully")
        print(df_old.head())
    else:
        print("Failed to fetch data")


Authenticated successfully
Opened Google Sheet: Tracking Sheet for Charlotte.xlsx
Accessing worksheet: Test
Fetched data from worksheet
Converted data to DataFrame
Data fetched successfully
   SNo      Version Sheet.Stage  Date tracking Enter      Invoice  \
0    1                  TRACK 2  2024-03-01 14:46:27  UK19670|OAU   
1    2  CANCELLED RAPTOR ORDERS  2024-03-02 17:17:21  UK20004|QCD   
2    3                  TRACK 3  2024-03-07 04:31:08  NL10750|QCD   
3   51                  TRACK 3  2024-04-05 09:39:51  NL10750|QCD   
4  767                  TRACK 3  2024-06-18 19:11:34  NL10750|QCD   

  Supplier Name Supplier (Grainger / Non-Grainger)  \
0    KSC DIRECT                         KSC DIRECT   
1          ebay                  ( cancelled) ebay   
2     indsencon        indsencon + andersonprocess   
3      anderson        indsencon + andersonprocess   
4     indsencon        indsencon + andersonprocess   

  Version Sheet.Placed the Order with Supplier         Tracking Number

In [2]:
df_old

,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
0,1,TRACK 2,2024-03-01 14:46:27,UK19670|OAU,KSC DIRECT,KSC DIRECT,Yes,1ZRF58610394781994,,UPS,...,PAID,15/02/2024,Flat Shipping (5 - 7 days),15/02/2024,Yes,No,,,,
1,2,CANCELLED RAPTOR ORDERS,2024-03-02 17:17:21,UK20004|QCD,ebay,( cancelled) ebay,Yes,9405508205499844047803,,USPS,...,NOT PAID,,2-3 WEEKS,28/02/2024,No,No,,,,
2,3,TRACK 3,2024-03-07 04:31:08,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204970878331,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
3,51,TRACK 3,2024-04-05 09:39:51,NL10750|QCD,anderson,indsencon + andersonprocess,Multiple Vendors,1z4e93640372781255,,UPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
4,767,TRACK 3,2024-06-18 19:11:34,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204137800311,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,972,TRACK 1,2024-06-22 16:51:47,AU4645|QCD,zoro,zoro,Yes,401265252128,2024-06-24T14:15:00-04:00,FEDEX,...,NOT PAID,,7-10 DAYS,21/06/2024,Yes,No,,,,
972,973,TRACK 1,2024-06-22 16:53:30,UK23368|OAD,zoro,zoro,Yes,401265249739,2024-06-24T14:15:00-04:00,FEDEX,...,PAID,20/06/2024,Flat Rate Shipping (5 - 7 days),20/06/2024,Yes,No,,,,
973,974,TRACK 1,2024-06-22 16:54:14,UK22937|QAD,zoro,Zoro,Yes,401265253146,2024-06-24T14:15:00-04:00,FEDEX,...,PAID,17/06/2024,8 DAYS,07/06/2024,Yes,No,,,,
974,975,TRACK 1,2024-06-22 16:54:59,UK23392|OAD,zoro,zoro,Yes,401265256682,2024-06-24T14:15:00-04:00,FEDEX,...,PAID,21/06/2024,Flat Rate Shipping (5 - 7 days),21/06/2024,No,No,,,,


In [3]:
import pandas as pd
from datetime import datetime, timedelta

# Ensure 'Date tracking Enter' is in datetime format
df_old['Date tracking Enter'] = pd.to_datetime(df_old['Date tracking Enter'])

# Get today's date
today = datetime.today()

# Calculate the date 7 days ago
seven_days_ago = today - timedelta(days=7)

# Filter the DataFrame for records from the last 7 days
df_last_7_days = df_old[df_old['Date tracking Enter'] >= seven_days_ago]

# Display the filtered DataFrame
df_last_7_days


,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
4,767,TRACK 3,2024-06-18 19:11:34,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204137800311,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
21,952,TRACK 1,2024-06-21 20:02:10,UK20279|QCD,MCM PO UK20279(4),MCM,Multiple Vendors,1Z0835200377901977,,UPS,...,NOT PAID,,7-10 DAYS,08/03/2024,Yes,No,,,,
32,815,TRACK 1,2024-06-19 16:36:37,UK17236|QAX,KSC Direct,KSC Direct,Yes,1Z4560160326196011,,UPS,...,PAID,22/11/2023,09-11 Weeks,30/10/2023,Yes,Yes,,,,
63,716,TRACK 1,2024-06-17 15:40:07,UK20441|QCD,Grainger,zoro ( 2 units ) + grainger ( 15 units) ( qou...,Multiple Vendors,402458417555,,FEDEX,...,NOT PAID,,5-8 WEEKS,14/03/2024,No,No,,,,
64,929,TRACK 1,2024-06-21 15:55:11,UK20441|QCD,Grainger,zoro ( 2 units ) + grainger ( 15 units) ( qou...,Multiple Vendors,403789183621,,FEDEX,...,NOT PAID,,5-8 WEEKS,14/03/2024,No,No,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,972,TRACK 1,2024-06-22 16:51:47,AU4645|QCD,zoro,zoro,Yes,401265252128,2024-06-24T14:15:00-04:00,FEDEX,...,NOT PAID,,7-10 DAYS,21/06/2024,Yes,No,,,,
972,973,TRACK 1,2024-06-22 16:53:30,UK23368|OAD,zoro,zoro,Yes,401265249739,2024-06-24T14:15:00-04:00,FEDEX,...,PAID,20/06/2024,Flat Rate Shipping (5 - 7 days),20/06/2024,Yes,No,,,,
973,974,TRACK 1,2024-06-22 16:54:14,UK22937|QAD,zoro,Zoro,Yes,401265253146,2024-06-24T14:15:00-04:00,FEDEX,...,PAID,17/06/2024,8 DAYS,07/06/2024,Yes,No,,,,
974,975,TRACK 1,2024-06-22 16:54:59,UK23392|OAD,zoro,zoro,Yes,401265256682,2024-06-24T14:15:00-04:00,FEDEX,...,PAID,21/06/2024,Flat Rate Shipping (5 - 7 days),21/06/2024,No,No,,,,


In [4]:
new_rows_df=df_last_7_days

In [5]:
import pandas as pd
import requests

# Define functions to interact with AfterShip API
def add_tracking(api_key, tracking_number, courier):
    url = "https://api.aftership.com/v4/trackings"
    headers = {
        "Content-Type": "application/json",
        "aftership-api-key": api_key
    }
    payload = {
        "tracking": {
            "tracking_number": tracking_number,
            "slug": courier
        }
    }
    
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 201:
        print(f"Tracking added successfully for {tracking_number}")
        return True
    elif response.status_code == 400 and "Tracking already exists" in response.text:
        print(f"Tracking already exists for {tracking_number}")
        return True
    else:
        print(f"Error adding tracking: {response.status_code}, {response.text}")
        return False

def get_estimated_delivery_date(api_key, tracking_number, courier):
    url = f"https://api.aftership.com/v4/trackings/{courier}/{tracking_number}"
    headers = {
        "Content-Type": "application/json",
        "aftership-api-key": api_key
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        tracking_info = response.json()
        if 'tracking' in tracking_info['data'] and 'expected_delivery' in tracking_info['data']['tracking']:
            return tracking_info['data']['tracking']['expected_delivery']
        elif 'tracking' in tracking_info['data'] and 'delivery_date' in tracking_info['data']['tracking']:
            return tracking_info['data']['tracking']['delivery_date']
        else:
            return "Estimated delivery date not available"
    else:
        print(f"Error retrieving tracking info: {response.status_code}, {response.text}")
        return None

# Example usage with DataFrame
api_key = "asat_a298ba22dccd4a53a9b3ec292ec888a2"

# data = {
#     "Tracking Number": ["1Z452A950334412720", "1Z901W400360355277", "9405511105501887836564"],
#     "Tracking Courier Details.Courier API List": ["UPS", "UPS", "fedex"],
#     "Delivery Date": [None, None, None]
# }

# new_rows_df = pd.DataFrame(data)

# Filter out rows where 'Tracking Courier Details.Courier API List' contains 'fedex' and convert remaining to lowercase
filtered_df = new_rows_df[~new_rows_df['Tracking Courier Details.Courier  API List'].str.contains('fedex', case=False)]
filtered_df['Tracking Courier Details.Courier  API List'] = filtered_df['Tracking Courier Details.Courier  API List'].str.lower()

# Loop through each row in the filtered DataFrame
for index, row in filtered_df.iterrows():
    tracking_number = row["Tracking Number"]
    courier = row["Tracking Courier Details.Courier  API List"]
    
    # Step 1: Add tracking to AfterShip
    if add_tracking(api_key, tracking_number, courier):
        # Step 2: Retrieve estimated delivery date
        estimated_delivery_date = get_estimated_delivery_date(api_key, tracking_number, courier)
        new_rows_df.loc[new_rows_df["Tracking Number"] == tracking_number, "Delivery Date"] = estimated_delivery_date

# Display the updated DataFrame
print(new_rows_df)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_3076\2456770014.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Tracking Courier Details.Courier  API List'] = filtered_df['Tracking Courier Details.Courier  API List'].str.lower()


Tracking already exists for 9434611206204137800311
Tracking already exists for 1Z0835200377901977
Tracking already exists for 1Z4560160326196011
Tracking already exists for 1Z35X5620354127056
Tracking already exists for TBA313917781866
Tracking already exists for 1Z1857300364813400
Tracking already exists for 1ZA874080344621468
Tracking already exists for 1Z0A91600396191842
Tracking already exists for TBA313855661804
Tracking already exists for 1Z55Y2820345920957
Tracking already exists for 1Z3817790349909265
Tracking already exists for 9400111105501869605529
Tracking already exists for 1ZE3549B0297295384
Tracking already exists for 1Z6818730384627849
Tracking already exists for 1Z0835200377794905
Tracking already exists for 1Z6544461331012251 ;1Z05W34E0334737936 ; 1Z6R8V320303751461
Tracking already exists for TBA313833542672
Tracking already exists for 1Z1043270396610615
Tracking already exists for TBA313846709829
Tracking already exists for 9361289745032563206847
Tracking already ex

In [7]:
new_rows_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291 entries, 4 to 975
Data columns (total 23 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   SNo                                            291 non-null    object        
 1   Version Sheet.Stage                            291 non-null    object        
 2   Date tracking Enter                            291 non-null    datetime64[ns]
 3   Invoice                                        291 non-null    object        
 4   Supplier Name                                  291 non-null    object        
 5   Supplier (Grainger / Non-Grainger)             291 non-null    object        
 6   Version Sheet.Placed the Order with Supplier   291 non-null    object        
 7   Tracking Number                                291 non-null    object        
 8   Delivery Date                                  180 non-null    ob

In [8]:
# import pandas as pd
# import gspread
# import time
# from oauth2client.service_account import ServiceAccountCredentials

# # Function to authenticate and get the Google Sheets client
# def authenticate_google_sheets(json_credentials_path):
#     scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
#     creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
#     client = gspread.authorize(creds)
#     return client

# # Function to fetch data from a worksheet and convert it to a DataFrame
# def fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path):
#     try:
#         # Authenticate and get the Google Sheets client
#         client = authenticate_google_sheets(json_credentials_path)
#         print("Authenticated successfully")

#         # Open the existing Google Sheet by name
#         sheet = client.open(sheet_name)
#         print(f"Opened Google Sheet: {sheet_name}")

#         # Get the worksheet by name
#         worksheet = sheet.worksheet(worksheet_name)
#         print(f"Accessing worksheet: {worksheet_name}")

#         # Fetch all data from the worksheet
#         data = worksheet.get_all_values()
#         print(f"Fetched data from worksheet: {data[:5]}")  # Print first 5 rows for debugging

#         # Check if data is not empty
#         if not data:
#             print("No data found in the worksheet")
#             return None, None

#         # Convert the data to a DataFrame
#         df_old = pd.DataFrame(data[1:], columns=data[0])
#         print(f"Converted data to DataFrame: {df_old.head()}")

#         return df_old, worksheet

#     except gspread.exceptions.APIError as api_error:
#         print(f"Google Sheets API error: {api_error}")
#         return None, None
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")
#         return None, None

# # Function to update the delivery dates in the Google Sheet
# def update_delivery_dates(new_rows_df, df_old, worksheet):
#     try:
#         # Ensure columns are correctly aligned
#         new_rows_df = new_rows_df[['Delivery Date', 'Tracking Number']]
#         print(f"new_rows_df: {new_rows_df.head()}")

#         # Map tracking numbers to delivery dates
#         tracking_to_delivery = dict(zip(df_old['Tracking Number'], df_old['Delivery Date']))
#         print(f"Tracking to Delivery Map: {tracking_to_delivery}")

#         # Update the delivery dates in the new_rows_df DataFrame
#         new_rows_df.loc[:, 'Delivery Date'] = new_rows_df['Tracking Number'].map(tracking_to_delivery)
#         print(f"Updated new_rows_df: {new_rows_df.head()}")

#         # Find the column index for 'Delivery Date' in the worksheet
#         header_row = worksheet.row_values(1)
#         delivery_date_col_index = header_row.index('Delivery Date') + 1

#         # Update the worksheet with the new delivery dates
#         for idx, row in new_rows_df.iterrows():
#             if pd.notnull(row['Delivery Date']):  # Only update if the delivery date is not null
#                 worksheet.update_cell(idx + 2, delivery_date_col_index, row['Delivery Date'])
#                 time.sleep(1)  # Introduce a delay to avoid hitting the write quota

#         print("Delivery dates updated successfully")
#     except Exception as e:
#         print(f"An unexpected error occurred while updating delivery dates: {e}")

# # Example usage
# if __name__ == "__main__":
#     # Path to the JSON credentials file
#     json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

#     # Name of the existing Google Sheet
#     sheet_name = 'Tracking Sheet for Charlotte.xlsx'

#     # Name of the worksheet
#     worksheet_name = 'Test'

#     # Fetch data from the worksheet into a DataFrame
#     df_old, worksheet = fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path)

#     if df_old is not None and worksheet is not None:
#         print("Data fetched successfully from Google Sheets")
#         print(df_old.head())  # Print the first few rows of df_old for debugging

#         # Sample DataFrame with new delivery dates
#         # new_rows_df = pd.DataFrame({
#         #     'Tracking Number': ['123', '456', '789'],
#         #     'Delivery Date': ['2023-06-01', '2023-06-02', '2023-06-03']
#         # })

#         # Authenticate and get the Google Sheets client
#         client = authenticate_google_sheets(json_credentials_path)
#         sheet = client.open(sheet_name)
#         worksheet = sheet.worksheet(worksheet_name)

#         # Update delivery dates in the Google Sheet
#         update_delivery_dates(new_rows_df, df_old, worksheet)
#     else:
#         print("Failed to fetch data")


Authenticated successfully
Opened Google Sheet: Tracking Sheet for Charlotte.xlsx
Accessing worksheet: Test
Fetched data from worksheet: [['SNo', 'Version Sheet.Stage', 'Date tracking Enter', 'Invoice', 'Supplier Name', 'Supplier (Grainger / Non-Grainger)', 'Version Sheet.Placed the Order with Supplier', 'Tracking Number', 'Delivery Date', 'Tracking Courier Details.Courier  API List', 'Tracking Courier Details.Tracking Destination', 'Version Sheet.Destination Point', 'Purchase Cost', 'Version Sheet.Order Payment Received Status', 'Version Sheet.Date of Funds Received', 'Version Sheet.Expected Shipment Date', 'Version Sheet.Date of Order Received', 'Pic', 'Item Pics', 'Weight_LBS', 'Dimensions_Inches', 'Remark by Robert', 'Remark by Logistic team'], ['1', 'TRACK 2', '2024-03-01 14:46:27', 'UK19670|OAU', 'KSC DIRECT', 'KSC DIRECT', 'Yes', '1ZRF58610394781994', '', 'UPS', 'Track 1', 'DAP GERMANY', '128.63', 'PAID', '15/02/2024', 'Flat Shipping (5 - 7 days)', '15/02/2024', 'Yes', 'No', '',


KeyboardInterrupt



In [13]:
new_rows_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291 entries, 4 to 975
Data columns (total 23 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   SNo                                            291 non-null    object        
 1   Version Sheet.Stage                            291 non-null    object        
 2   Date tracking Enter                            291 non-null    datetime64[ns]
 3   Invoice                                        291 non-null    object        
 4   Supplier Name                                  291 non-null    object        
 5   Supplier (Grainger / Non-Grainger)             291 non-null    object        
 6   Version Sheet.Placed the Order with Supplier   291 non-null    object        
 7   Tracking Number                                291 non-null    object        
 8   Delivery Date                                  180 non-null    ob

In [18]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Function to authenticate and get the Google Sheets client
def authenticate_google_sheets(json_credentials_path):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)
    return client

# Function to fetch data from a worksheet and convert it to a DataFrame
def fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path):
    try:
        # Authenticate and get the Google Sheets client
        client = authenticate_google_sheets(json_credentials_path)
        print("Authenticated successfully")

        # Open the existing Google Sheet by name
        sheet = client.open(sheet_name)
        print(f"Opened Google Sheet: {sheet_name}")

        # Get the worksheet by name
        worksheet = sheet.worksheet(worksheet_name)
        print(f"Accessing worksheet: {worksheet_name}")

        # Fetch all data from the worksheet
        data = worksheet.get_all_values()
        print(f"Fetched data from worksheet: {data[:5]}")  # Print first 5 rows for debugging

        # Check if data is not empty
        if not data:
            print("No data found in the worksheet")
            return None, None

        # Convert the data to a DataFrame
        df_old = pd.DataFrame(data[1:], columns=data[0])
        print(f"Converted data to DataFrame: {df_old.head()}")

        return df_old, worksheet

    except gspread.exceptions.APIError as api_error:
        print(f"Google Sheets API error: {api_error}")
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None

# Function to update the delivery dates in the Google Sheet
def update_delivery_dates(new_rows_df, df_old, worksheet):
    try:
        # Ensure only the necessary columns are used for update
        new_rows_df = new_rows_df[['Tracking Number', 'Delivery Date']]
        print(f"new_rows_df: {new_rows_df.head()}")

        # Remove duplicates by taking the last occurrence of each Tracking Number
        new_rows_df = new_rows_df.drop_duplicates(subset='Tracking Number', keep='last')

        # Set index for merging
        df_old.set_index('Tracking Number', inplace=True)
        new_rows_df.set_index('Tracking Number', inplace=True)

        # Update the 'Delivery Date' in the original DataFrame
        df_old.update(new_rows_df)
        df_old.reset_index(inplace=True)
        print(f"Updated df_old: {df_old.head()}")

        # Convert the updated DataFrame back to a list of lists
        updated_data = [df_old.columns.values.tolist()] + df_old.values.tolist()

        # Clear the existing worksheet
        worksheet.clear()

        # Update the worksheet with the new data
        worksheet.update(updated_data)
        print("Delivery dates updated successfully")

    except Exception as e:
        print(f"An unexpected error occurred while updating delivery dates: {e}")

# Example usage
if __name__ == "__main__":
    # Path to the JSON credentials file
    json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

    # Name of the existing Google Sheet
    sheet_name = 'Tracking Sheet for Charlotte.xlsx'

    # Name of the worksheet
    worksheet_name = 'Test'

    # Fetch data from the worksheet into a DataFrame
    df_old, worksheet = fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path)

    if df_old is not None and worksheet is not None:
        print("Data fetched successfully from Google Sheets")
        print(df_old.head())  # Print the first few rows of df_old for debugging

        # Sample DataFrame with new delivery dates (Replace with your actual data)
        # new_rows_df = pd.DataFrame({
        #     'SNo': [1, 2, 3],
        #     'Version Sheet.Stage': ['Stage1', 'Stage2', 'Stage3'],
        #     'Date tracking Enter': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Invoice': ['Inv1', 'Inv2', 'Inv3'],
        #     'Supplier Name': ['Supplier1', 'Supplier2', 'Supplier3'],
        #     'Supplier (Grainger / Non-Grainger)': ['Grainger', 'Non-Grainger', 'Grainger'],
        #     'Version Sheet.Placed the Order with Supplier': ['Yes', 'No', 'Yes'],
        #     'Tracking Number': ['123', '456', '789'],
        #     'Delivery Date': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Tracking Courier Details.Courier  API List': ['API1', 'API2', 'API3'],
        #     'Tracking Courier Details.Tracking Destination': ['Dest1', 'Dest2', 'Dest3'],
        #     'Version Sheet.Destination Point': ['Point1', 'Point2', 'Point3'],
        #     'Purchase Cost': ['Cost1', 'Cost2', 'Cost3'],
        #     'Version Sheet.Order Payment Received Status': ['Received', 'Pending', 'Received'],
        #     'Version Sheet.Date of Funds Received': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Version Sheet.Expected Shipment Date': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Version Sheet.Date of Order Received': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Pic': ['Pic1', 'Pic2', 'Pic3'],
        #     'Item Pics': ['ItemPic1', 'ItemPic2', 'ItemPic3'],
        #     'Weight_LBS': ['10', '20', '30'],
        #     'Dimensions_Inches': ['10x10x10', '20x20x20', '30x30x30'],
        #     'Remark by Robert': ['Remark1', 'Remark2', 'Remark3'],
        #     'Remark by Logistic team': ['LogisticRemark1', 'LogisticRemark2', 'LogisticRemark3']
        # })

        # Update delivery dates in the Google Sheet
        update_delivery_dates(new_rows_df, df_old, worksheet)
    else:
        print("Failed to fetch data")


Authenticated successfully
Opened Google Sheet: Tracking Sheet for Charlotte.xlsx
Accessing worksheet: Test
Fetched data from worksheet: [['SNo', 'Version Sheet.Stage', 'Date tracking Enter', 'Invoice', 'Supplier Name', 'Supplier (Grainger / Non-Grainger)', 'Version Sheet.Placed the Order with Supplier', 'Tracking Number', 'Delivery Date', 'Tracking Courier Details.Courier  API List', 'Tracking Courier Details.Tracking Destination', 'Version Sheet.Destination Point', 'Purchase Cost', 'Version Sheet.Order Payment Received Status', 'Version Sheet.Date of Funds Received', 'Version Sheet.Expected Shipment Date', 'Version Sheet.Date of Order Received', 'Pic', 'Item Pics', 'Weight_LBS', 'Dimensions_Inches', 'Remark by Robert', 'Remark by Logistic team'], ['1', 'TRACK 2', '2024-03-01 14:46:27', 'UK19670|OAU', 'KSC DIRECT', 'KSC DIRECT', 'Yes', '1ZRF58610394781994', '', 'UPS', 'Track 1', 'DAP GERMANY', '128.63', 'PAID', '15/02/2024', 'Flat Shipping (5 - 7 days)', '15/02/2024', 'Yes', 'No', '',